In [6]:
import cv2
import pandas as pd
import os

In [8]:
def convertToRGB(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [9]:
def detect_face(img):
#convert the test image to gray scale as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
#load OpenCV face detector, I am using LBP which is fast
#there is also a more accurate but slow: Haar classifier
    face_cascade = cv2.CascadeClassifier('/Users/jun/Downloads/FaceDetect-master/haarcascade_frontalface_default.xml')
 
#let's detect multiscale images(some images may be closer to camera than others)
#result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
 

    if (len(faces) == 0):
        return None, None
 
#under the assumption that there will be only one face,
#extract the face area
    (x, y, w, h) = faces[0]
 
#return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [10]:
import numpy as np
import pandas as pd

In [11]:
def prepare_training_data(data_folder_path):
 
#------STEP-1--------
#get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
 
#list to hold all subject faces
    faces = []
#list to hold labels for all subjects
    labels = []
    for dir_name in dirs:
        if dir_name.startswith("."):
            continue;
        label = dir_name
 
#build path of directory containing images for current subject subject
#sample subject_dir_path = "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name
 
#get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
 
#------STEP-3--------
#go through each image name, read image, 
#detect face and add face to list of faces
        for image_name in subject_images_names:
 
#ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
 
#build image path
#sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name
 
#read image
            image = cv2.imread(image_path)
 
#display an image window to show the image 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
 
#detect face
            face, rect = detect_face(image)
 
#------STEP-4--------
#for the purpose of this tutorial
#we will ignore faces that are not detected
            if face is not None:
#add face to list of faces
                faces.append(face)
#add label for this face
                labels.append(label)
 
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        cv2.destroyAllWindows()
 
    return faces, labels

In [12]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and the other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("/Users/jun/Downloads/lfw2")
print("Data prepared")

print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  111
Total labels:  111


In [13]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

In [17]:
#train our face recognizer of our training faces
label_2 = [i for i in range(len(labels))]
face_recognizer.train(faces, np.array(label_2))

subjects = labels


In [22]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)    

In [26]:
def predict(test_img):
#make a copy of the image as we don't want to change original image
    img = test_img.copy()
#detect face from the image
    face, rect = detect_face(img)
 
#predict the image using our face recognizer 
    label, conf = face_recognizer.predict(face)
#get name of respective label returned by face recognizer
    label_text = subjects[label]
#draw a rectangle around face detected
    draw_rectangle(img, rect)
#draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)

    return img, label


In [30]:
test_img1 = cv2.imread("/Users/jun/Downloads/lfw2/Aaron_Tippin/Aaron_Tippin_0001.jpg")
test_img2 = cv2.imread("/Users/jun/stuff/Photo_yoon.png")

predicted_img1, img1_label = predict(test_img1)
predicted_img2, img2_label = predict(test_img2)
cv2.imshow(subjects[img1_label], predicted_img1)
cv2.imshow(subjects[img2_label], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [31]:
cv2.waitKey(1000)
cv2.destroyAllWindows()